<a href="https://colab.research.google.com/github/sashabel1/Cloud-tirgul/blob/master/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# התקנת ספריית MQTT
!pip install paho-mqtt

# ייבוא ספריות
import json
import time
import threading
import paho.mqtt.client as mqtt

# רשימת הודעות שהתקבלו
received_messages = []

# פונקציית callback שמופעלת עם קבלת הודעה
def on_message(client, userdata, msg):
    try:
        payload = json.loads(msg.payload.decode())
        topic = msg.topic
        print(f"\n📡 Topic: {topic}")
        print("📦 Data:", payload)
        received_messages.append((topic, payload))
    except Exception as e:
        print("Error decoding message:", e)

# יצירת לקוח MQTT
client = mqtt.Client()

# חיבור הפונקציה לקבלת הודעות
client.on_message = on_message

# התחברות לשרת
client.connect("test.mosquitto.org", 1883, 60)

# הרשמה ל-topics
client.subscribe("braude/D106/indoor")
client.subscribe("braude/D106/outdoor")

# הפעלת הלולאה ברקע (כדי לא לחסום את הקולאב)
def start_loop():
    client.loop_forever()

thread = threading.Thread(target=start_loop)
thread.start()

# המתנה של 30 שניות כדי לראות הודעות נכנסות
print("⏳ מחכה לקבלת נתונים...")
time.sleep(30)

# ניתוק הלקוח
client.disconnect()
print("\n✅ סיום ההאזנה.")

# פונקציית מנוע חיפוש
def search_data(keyword=None, min_temp=None, max_temp=None, topic_filter=None):
    print("\n🔎 תוצאות חיפוש:")
    for topic, data in received_messages:
        # סינון לפי topic
        if topic_filter and topic_filter not in topic:
            continue

        # סינון לפי טמפרטורה
        temp = data.get("Temperature")
        if min_temp is not None and (temp is None or temp < min_temp):
            continue
        if max_temp is not None and (temp is None or temp > max_temp):
            continue

        # סינון לפי מילת מפתח כלשהי (לדוגמה "Humidity" או ערך)
        if keyword:
            if not any(keyword.lower() in str(value).lower() for value in data.values()):
                continue

        print(f"🌐 Topic: {topic} | 📦 {data}")

# דוגמאות לחיפוש:
search_data(min_temp=25, max_temp=27, topic_filter="indoor")
# או למשל:
# search_data(keyword="Lux", topic_filter="outdoor")


In [1]:
# התקנת ספריית MQTT
!pip install paho-mqtt

# ייבוא ספריות
import json
import time
import paho.mqtt.client as mqtt
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output

# משתנה גלובלי לשמירת הודעות שהתקבלו
received_messages = []

# פונקציה שמופעלת בעת קבלת הודעה
def on_message(client, userdata, msg):
    try:
        payload = json.loads(msg.payload.decode())
        topic = msg.topic
        received_messages.append((topic, payload))
    except Exception as e:
        print("Error decoding message:", e)

# פונקציית מנוע חיפוש – סינון לפי מילת מפתח
def search_data(keyword):
    print("\n🔍results for:", keyword)
    found = False
    for topic, data in received_messages:
        # בדיקה אם המילה קיימת בשם מדד או בערכים
        if keyword.lower() in topic.lower() or \
           any(keyword.lower() in str(k).lower() or keyword.lower() in str(v).lower() for k, v in data.items()):
            print(f"\n🌐 Topic: {topic}")
            print("📦 data:", data)
            found = True
    if not found:
        print("no results😔")

# פונקציית הפעלה מלאה
def run_search(b):
    received_messages.clear()  # ניקוי תוצאות ישנות
    keyword = keyword_input.value.strip()
    if not keyword:
        print("must enter value❗")
        return

    # יצירת לקוח MQTT
    client = mqtt.Client()
    client.on_message = on_message
    client.connect("test.mosquitto.org", 1883, 60)
    client.subscribe("braude/D106/indoor")
    client.subscribe("braude/D106/outdoor")
    clear_output(wait=True)

    print(f"🔎Searching for'{keyword}' listening for 30 seconds")
    client.loop_start()
    time.sleep(30)
    client.loop_stop()
    client.disconnect()

    # חיפוש בתוצאות
    search_data(keyword)

# יצירת ממשק משתמש
keyword_input = widgets.Text(
    value='',
    placeholder='Enter: Temperature, Humidity, Lux, 974',
    description='search:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

search_button = widgets.Button(
    description='search',
    button_style='success'
)

search_button.on_click(run_search)

# הצגת ממשק
display(keyword_input, search_button)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.9 MB/s eta 0:00:00


Text(value='', description='חיפוש:', layout=Layout(width='80%'), placeholder='לדוגמה: Temperature, Humidity, L…

Button(button_style='success', description='התחל חיפוש', style=ButtonStyle())

<ipython-input-1-d3de72518426>:46: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()



🔎 מחפש את 'Temperture' ... מאזין לנתונים ל-30 שניות...

🔍 תוצאות עבור: Temperture
לא נמצאו תוצאות 😔


<ipython-input-1-d3de72518426>:46: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()



🔎 מחפש את 'Temperature' ... מאזין לנתונים ל-30 שניות...

🔍 תוצאות עבור: Temperature

🌐 Topic: braude/D106/outdoor
📦 נתונים: {'Dlight': 73.13, 'Temperature': 19.38, 'Humidity': 73.15}

🌐 Topic: braude/D106/outdoor
📦 נתונים: {'Dlight': 73.12, 'Temperature': 19.38, 'Humidity': 73.11}

🌐 Topic: braude/D106/outdoor
📦 נתונים: {'Dlight': 73.09, 'Temperature': 19.37, 'Humidity': 73.1}

🌐 Topic: braude/D106/indoor
📦 נתונים: {'Humidity': 51.64, 'Temperature': 25.64, 'Pressure': 971.28, 'Distance': 926.16}

🌐 Topic: braude/D106/indoor
📦 נתונים: {'Humidity': 51.6, 'Temperature': 25.62, 'Pressure': 971.27, 'Distance': 926.84}

🌐 Topic: braude/D106/indoor
📦 נתונים: {'Humidity': 51.58, 'Temperature': 25.64, 'Pressure': 971.2399, 'Distance': 922.76}

🌐 Topic: braude/D106/indoor
📦 נתונים: {'Humidity': 51.56, 'Temperature': 25.61, 'Pressure': 971.3099, 'Distance': 922.76}

🌐 Topic: braude/D106/indoor
📦 נתונים: {'Humidity': 51.49, 'Temperature': 25.62, 'Pressure': 971.2299, 'Distance': 926.84}

🌐 Topic: